In [118]:
import os
import numpy as np # type: ignore
import re


class CogBeaconDataset:
    def __init__(self, root_path):
        self.root_path = root_path

    def parse_folder_name(self, folder_name): # this is for reading the session folder that contains eeg files
        parts = folder_name.split('_')
        user_id = parts[1]
        stimuli_type = parts[2]
        game_mode = parts[3]
        
        if "b" in user_id:
            session_day = "Second"
            wcst_version = "V2" if game_mode == "m" else "o"
        else:
            session_day = "First"
            wcst_version = "V1" if game_mode == "m" else "o"
        
        return {
            'user_id': user_id,
            'stimuli_type': stimuli_type,
            'game_mode': game_mode,
            'session_day': session_day,
            'wcst_version': wcst_version
        }

    def parse_file_name(self, file_name): # this for each eeg data in easch session
        parts = file_name.split('_')
        round_id_same_rule = parts[0] # the same ids of the i=first part is the when the rounds in the game have the same rule to follow
        round_id = parts[1].split('.')[0] 
        
        return {
            'round_id_same_rule': round_id_same_rule,
            'round_id': round_id
        }
    
    def read_eeg_file(self, file_path):
        eeg_data = []
        try:
            with open(file_path, 'r') as file:
                for line in file:
                    if line.startswith('eeg'):
                        data = list(map(float, line.split()[1:]))
                        eeg_data.append(data)
            return eeg_data
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")
            return None

    def process_session(self, session_folder):
        session_path = os.path.join(self.root_path, session_folder)
        session_info = self.parse_folder_name(session_folder)
        
        eeg_data = []
        
        file_names = os.listdir(session_path)

        def sort_key(file_name): # sorting the filename to make the reading of the data in the order of the expriment time
            match = re.match(r"(\d+)_(\d+)", file_name)
            if match:
                return (int(match.group(1)), int(match.group(2)))  
            return (float('inf'), float('inf')) 
        file_names.sort(key=sort_key)

        for file_name in file_names:
            # print("process session", file_name)
            file_info = self.parse_file_name(file_name)
            file_path = os.path.join(session_path, file_name)
            data = self.read_eeg_file(file_path)
            eeg_data.append(data) 
        return eeg_data
                # eeg_data.append({
                #     'file_info': file_info,
                #     'data': data
                # })
        
        # return {
        #     'session_info': session_info,
        #     'eeg_data': eeg_data
        # }
        # return eeg_data

    def load_all_sessions(self):
        all_sessions = []
        for session_folder in os.listdir(self.root_path):
            if os.path.isdir(os.path.join(self.root_path, session_folder)):
                session_data = self.process_session(session_folder)
                all_sessions.append(session_data)
        return all_sessions
    
    def load_session_by_participant(self, user_id, session_day,game_mode,channel):
        # target_sessions = []
        for session_folder in os.listdir(self.root_path):
            if os.path.isdir(os.path.join(self.root_path, session_folder)):
                session_info = self.parse_folder_name(session_folder)
                if session_info['user_id'] == user_id and session_info['session_day'] == session_day and session_info['game_mode']  == game_mode:
                    session_data = self.process_session(session_folder)
                    print("session_data", len(session_data),session_data[0])
                    # target_sessions.append(session_data)
        if channel:
            session_data = self.separet_channels_per_signal(session_data)[channel]
            print(len(session_data),session_data[0])
              
        return session_data

    def separet_channels_per_signal(self,signal):
        #  "TP9 FP1 FP2 TP10"
        TP9, FP1,FP2,TP10 = [],[],[],[]
        # print(signal)
        for sig in signal:
            for rnd in sig:
                TP9.append(rnd[0])
                FP1.append(rnd[1])
                FP2.append(rnd[2])
                TP10.append(rnd[3])
        return {
                "TP9":TP9,
                "FP1":FP1,
                "FP2":FP2,
                "TP10":TP10
            }


In [119]:
    
cogbeacon_root_path = '/Users/athenasaghi/Desktop/Datasets/CogBeacon-MultiModal_Dataset_for_Cognitive_Fatigue-master/'
cogbeacon_eeg_path = os.path.join(cogbeacon_root_path, 'eeg')
dataset = CogBeaconDataset(cogbeacon_eeg_path)


In [116]:
user_id = '0'
session_day = 'First'
game_mode = 'm'
participant_data = dataset.load_session_by_participant(user_id, session_day,game_mode)


session_data 120 [[819.200195312, 815.91027832, 838.940002441, 832.360046387], [820.845214844, 819.200195312, 832.360046387, 837.29498291], [809.330322266, 817.555236816, 829.070068359, 827.425109863], [812.620300293, 820.845214844, 829.070068359, 830.715087891], [814.265258789, 822.49017334, 824.135131836, 834.005004883], [820.845214844, 815.91027832, 822.49017334, 840.584960938], [825.780151367, 817.555236816, 830.715087891, 843.874938965], [825.780151367, 815.91027832, 840.584960938, 829.070068359], [812.620300293, 810.975280762, 827.425109863, 825.780151367], [819.200195312, 807.68536377, 830.715087891, 825.780151367], [814.265258789, 807.68536377, 829.070068359, 830.715087891], [802.750427246, 809.330322266, 822.49017334, 819.200195312], [812.620300293, 819.200195312, 830.715087891, 822.49017334], [810.975280762, 815.91027832, 832.360046387, 820.845214844], [812.620300293, 806.040344238, 837.29498291, 819.200195312], [820.845214844, 799.460449219, 830.715087891, 834.005004883], [8

In [117]:
len(participant_data)

84495